In [ ]:
import polars as pl
import networkx as nx
import time
import rtsvg
from linknode_graph_patterns import LinkNodeGraphPatterns
rt = rtsvg.RACETrack()
def graphAsDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)
_patterns_ = LinkNodeGraphPatterns()
_tiles_    = []
for _type_ in _patterns_.types:
    g      = _patterns_.createPattern(_type_)
    _lms_  = rt.identifyLandmarks(g)
    _node_colors_ = {}
    for _lm_ in _lms_[0]: _node_colors_[_lm_] = 'red'
    df     = graphAsDataFrame(g)
    params = {'df':df, 'relationships':[('fm','to')], 'w':256, 'h':256, 'node_size':'small', 'link_arrow':False, 'node_color':_node_colors_}
    t0 = time.time()
    pos_nx = nx.spring_layout(g)
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_nx,  **params), f'Spring (NX) {t1-t0:.2f}s'))

    t0 = time.time()
    pos_ref = rt.springLayout(g)
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_ref, **params), f'Spring (RT) {t1-t0:.2f}s'))
    
    t0 = time.time()
    pos_psl = rtsvg.PolarsForceDirectedLayout(g, k=0).results()
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_psl, **params), f'Polar_K_0 (RT) {t1-t0:.2f}s'))
                   
    t0 = time.time()
    pos_psl = rtsvg.PolarsForceDirectedLayout(g, k=1).results()
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_psl, **params), f'Polar_K_1 (RT) {t1-t0:.2f}s'))
    
    t0 = time.time()
    pos_psl = rtsvg.PolarsForceDirectedLayout(g, k=2).results()
    t1 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(pos=pos_psl, **params), f'Polar_K_2 (RT) {t1-t0:.2f}s'))
rt.table(_tiles_, per_row=5, spacer=10)